In [ ]:
import os
import random
import numpy as np
import pandas as pd
import scipy as sp
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset,  random_split
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from parameter import *
from VAE_Model import *
#from denoise_model import DenoiseNN, p_losses, sample, linear_beta_schedule, cosine_beta_schedule, compute_total_mse
from denoise_model import DenoisingModel, diffusion_loss, sample_from_model, linear_beta_schedule, cosine_beta_schedule
import joblib


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Clear CUDA cache
torch.cuda.empty_cache()

# Set seeds for reproducibility
torch.manual_seed(0)
rng = np.random.default_rng()
np.random.seed(13)


In [ ]:
Properties = pd.read_csv('files/Data/Properties.csv')
Compositions = pd.read_csv('files/Data/Compositions.csv')

'''
Properties indices:
 0: 'Density'
 1: 'Young's modulus'
 2: 'Flexural modulus'
 3: 'Shear modulus'
 4: 'Bulk modulus'
 5: 'Poisson's ratio'
 6: 'Melting point'
 7: 'Thermal conductivity'
 8: 'Specific heat capacity'
 9: 'Thermal expansion coefficient'
10: 'Latent heat of fusion'
11: 'Electrical conductivity'
12: 'Acoustic velocity'
13: 'Average Atomic Weight'
'''

'''
Compositions indices:
  0: 'Ag (silver)'
  1: 'Al (aluminum)'
  2: 'As (arsenic)'
  3: 'Au (gold)'
  4: 'B (boron)'
  5: 'Be (beryllium)'
  6: 'BeO (beryllia)'
  7: 'Bi (bismuth)'
  8: 'C (carbon)'
  9: 'Ca (calcium)'
 10: 'Cd (cadmium)'
 11: 'Ce (cerium)'
 12: 'Co (cobalt)'
 13: 'Cr (chromium)'
 14: 'Cu (copper)'
 15: 'Dy (dysprosium)'
 16: 'Er (erbium)'
 17: 'Eu (europium)'
 18: 'Fe (iron)'
 19: 'Ga (gallium)'
 20: 'Gd (gadolinium)'
 21: 'Ge (germanium)'
 22: 'H (hydrogen)'
 23: 'Hf (hafnium)'
 24: 'Ho (holmium)'
 25: 'In (indium)'
 26: 'Ir (iridium)'
 27: 'La (lanthanum)'
 28: 'Li (lithium)'
 29: 'Lu (lutetium)'
 30: 'Mg (magnesium)'
 31: 'Mn (manganese)'
 32: 'Mo (molybdenum)'
 33: 'N (nitrogen)'
 34: 'Nb (niobium)'
 35: 'Nd (neodymium)'
 36: 'Ni (nickel)'
 37: 'O (oxygen)'
 38: 'O2 (oxygen gas)'
 39: 'Os (osmium)'
 40: 'P (phosphorus)'
 41: 'Pb (lead)'
 42: 'Pd (palladium)'
 43: 'Pr (praseodymium)'
 44: 'Pt (platinum)'
 45: 'Re (rhenium)'
 46: 'Rh (rhodium)'
 47: 'Ru (ruthenium)'
 48: 'S (sulfur)'
 49: 'Sb (antimony)'
 50: 'Sc (scandium)'
 51: 'Se (selenium)'
 52: 'Si (silicon)'
 53: 'Sm (samarium)'
 54: 'Sn (tin)'
 55: 'Sr (strontium)'
 56: 'Ta (tantalum)'
 57: 'Tb (terbium)'
 58: 'Te (tellurium)'
 59: 'ThO2 (thoria)'
 60: 'Ti (titanium)'
 61: 'Tl (thallium)'
 62: 'Tm (thulium)'
 63: 'U (uranium)'
 64: 'V (vanadium)'
 65: 'Y (Yttrium)'
 66: 'Yb (Ytterbium)'
 67: 'W (Tungsten)'
 68: 'Zn (Zinc)'
 69: 'Zr (Zirconium)'
'''


scaler_y = joblib.load('files/Scalers/scaler_Properties.pkl')
X = Compositions
Y = Properties

XX = X.values
YY = scaler_y.transform(Y)

In [ ]:
XX = torch.tensor(XX, dtype=torch.float32)
YY = torch.tensor(YY, dtype=torch.float32)

dataset  = TensorDataset(XX,YY)

num_train = len(dataset)
split = int(np.floor(valid_size * num_train)) 
train_dataset, test_dataset = random_split(dataset = dataset, lengths = [num_train - split,split])
train_loader = DataLoaderX(train_dataset, batch_size = batch_size, shuffle = True, pin_memory = True)
test_loader = DataLoaderX(test_dataset, batch_size = test_batch_size, shuffle = True, pin_memory = True)

In [ ]:
vae = vaeModel().to(device)
p_model = pModel().to(device)
vae.load_state_dict(torch.load(savedModelFolder + '/model.pt'))
p_model.load_state_dict(torch.load(savedModelFolder + '/p_model.pt'))


vae.eval()
p_model.eval()

In [ ]:
# Define beta schedule
betas = linear_beta_schedule(timesteps=timesteps)  # Or cosine_beta_schedule(timesteps=timesteps)

# Define alphas and related variables
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)

# Intermediate values for the model
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)

# Posterior variance computation
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

# Initialize DenoiseNN model
# Initialize DenoisingModel

denoise_model = DenoisingModel(
    z_dim=latent_dim,           # Dimensionality of noisy input
    hidden_dim=hidden_dim_denoise,  # Hidden layer size
    depth=n_layers_denoise,     # Number of MLP layers
    cond_in=n_properties,       # Conditioning input dimension
    cond_dim=dim_condition      # Projected conditioning vector dimension
).to(device)

# Optimizer and scheduler setup
optimizer = torch.optim.Adam(denoise_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.1)

# Print number of trainable parameters
trainable_params_diff = sum(p.numel() for p in denoise_model.parameters() if p.requires_grad)
print("Number of Diffusion model's trainable parameters: " + str(trainable_params_diff))




In [ ]:
if train_denoiser:
    # Train denoising model
    best_val_loss = np.inf
    for epoch in range(1, epochs_denoise + 1):
        denoise_model.train()
        train_loss_all = 0
        train_count = 0

        for data in train_loader:
            composition = data[0].to(device)
            properties = data[1].to(device)
            conditional_properties = properties[:, [0, 1, 5]]

            optimizer.zero_grad()
            with torch.no_grad():
                z_g, mu, std = vae.encoder(composition)

            t = torch.randint(0, timesteps, (z_g.size(0),), device=device).long()
            loss = diffusion_loss(denoise_model, z_g, t, conditional_properties,
                                  sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, loss_mode="l2")

            loss.backward()
            train_loss_all += z_g.size(0) * loss.item()
            train_count += z_g.size(0)
            optimizer.step()

        denoise_model.eval()
        val_loss_all = 0
        val_count = 0

        for data in test_loader:
            composition = data[0].to(device)
            properties = data[1].to(device)
            conditional_properties = properties[:, [0, 1, 5]]

            with torch.no_grad():
                z_g, mu, std = vae.encoder(composition)

            t = torch.randint(0, timesteps, (z_g.size(0),), device=device).long()
            loss = diffusion_loss(denoise_model, z_g, t, conditional_properties,
                                  sqrt_alphas_cumprod, sqrt_one_minus_alphas_cumprod, loss_mode="l2")

            val_loss_all += z_g.size(0) * loss.item()
            val_count += z_g.size(0)

        if epoch % 5 == 0:
            print(f"Epoch {epoch:03d} | Train Loss: {train_loss_all / train_count:.5f} | Val Loss: {val_loss_all / val_count:.5f}")

        scheduler.step()

        if best_val_loss >= val_loss_all:
            best_val_loss = val_loss_all
            torch.save({
                'state_dict': denoise_model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }, savedModelFolder + '/denoise_model1.pth.tar')

else:
    checkpoint = torch.load(savedModelFolder + '/denoise_model1.pth.tar')
    denoise_model.load_state_dict(checkpoint['state_dict'])

denoise_model.eval()


In [ ]:
checkpoint = torch.load(savedModelFolder +'/denoise_model.pth.tar')
denoise_model.load_state_dict(checkpoint['state_dict'])
denoise_model.eval()

In [ ]:
test_loader = DataLoaderX(test_dataset, batch_size = 1, shuffle = True, pin_memory = True)
train_loader = DataLoaderX(train_dataset, batch_size = 1, shuffle = True, pin_memory = True)

In [ ]:
# Inference step 

c_pred_all = []  # Store predicted properties
c_test_all = []  # Store ground-truth properties
refined_compositions_all = []  # Store best refined compositions

best_sample_pred = None

with torch.no_grad():
    for composition, properties in test_loader:
        composition = composition.to(device)  # Input composition 
        properties = properties.to(device)    # Ground truth material properties

        conditional_properties = properties[:, [0, 1, 5]]  # Selected conditional features
        y_cond = conditional_properties.repeat(50, 1).to(device)  # Expand for sampling

        batch_size = y_cond.size(0)

        # Generate samples from denoising model conditioned on selected properties
        samples = sample_from_model(
            denoise_model, y_cond,
            z_dim=latent_dim,
            n_steps=timesteps,
            beta_sched=betas,
            batch_sz=batch_size
        )
        samples_np = samples[-1]  # Get final time step 

        min_mse = float('inf')  # Initialize minimum MSE
        best_predicted_composition = None

        for sample0 in samples_np:
            sample0 = sample0.reshape(1, 15)  # Reshape latent vector

            # Decode latent vector into composition
            predicted_composition = vae.decoder(sample0)

            # Refine composition (round and redistribute tiny values)
            predicted_composition = refine_composition(predicted_composition)

            # Re-encode refined composition to get latent mean
            z_g, mean, std = vae.encoder(predicted_composition)

            # Predict properties using TensorFlow model
            y_predicted = p_model(mean)

            # Convert prediction to PyTorch tensor for MSE calculation
            y_predicted_tensor = torch.tensor(y_predicted, dtype=torch.float32, device=device)

            # Compute MSE between predicted and true conditional properties
            mse_value = compute_total_mse(
                properties.reshape(-1, 14)[:, [0, 1, 5]],
                y_predicted_tensor[:, [0, 1, 5]]
            )

            # Update best sample if current is better
            if mse_value < min_mse:
                min_mse = mse_value
                best_sample_pred = torch.tensor(sample0, device=device)
                best_y_predicted_tensor = y_predicted_tensor.clone()
                best_predicted_composition = predicted_composition.clone()

        # Save predictions and ground truth for the current batch
        c_pred_all.append(best_y_predicted_tensor.cpu().numpy())
        c_test_all.append(properties.cpu().detach().numpy())
        refined_compositions_all.append(best_predicted_composition.cpu().numpy())

# Concatenate all predictions and labels
c_pred_all = np.concatenate(c_pred_all, axis=0)
c_test_all = np.concatenate(c_test_all, axis=0)
refined_compositions_all = np.concatenate(refined_compositions_all, axis=0)

# Inverse scale the predicted and true properties
c_pred_all = scaler_y.inverse_transform(c_pred_all)
c_test_all = scaler_y.inverse_transform(c_test_all)